instalacion de prerequisitos

In [2]:
#pip install sagemaker_datawrangler

In [3]:
#pip install  opencv-python-headless

In [1]:
from tqdm import tqdm
import os
import cv2

In [2]:
import sagemaker_datawrangler           # For interactive data prep widget
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
import sagemaker
bucket=sagemaker.Session().default_bucket()

In [4]:
bucket # Bucket que se crea cuando creamos nuestro Dominio e iniciaos el Studio Een SageMaker

'sagemaker-us-east-2-244680720635'

In [5]:
prefix = 'mlops/tarea-2'


In [6]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [7]:
role

'arn:aws:iam::244680720635:role/service-role/AmazonSageMaker-ExecutionRole-20250331T182939'

Descarga de datos

In [8]:
#pip install kagglehub

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("danielshanbalico/dog-emotion")

print("Path to dataset files:", path)

Path to dataset files: /home/sagemaker-user/.cache/kagglehub/datasets/danielshanbalico/dog-emotion/versions/1


In [10]:
# Ruta del archivo de origen y destino
origen = path
destino = 'Datos'

# Comando cp
comando = f'cp -r {origen}* {destino}'

# Ejecutar el comando
os.system(comando)


0

Subir archivos a S3

In [ ]:
!wget https://github.com/velasquezjeisson/UdeMCloud/blob/main/script.py --no-check-certificate

In [11]:

def upload_folder_to_s3(bucket_name, folder_path, s3_folder_path):
    s3_client = boto3.client('s3')
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, folder_path)
            s3_path = os.path.join(s3_folder_path, relative_path)
            
            s3_client.upload_file(local_path, bucket_name, s3_path)
            print(f'Subido {local_path} a s3://{bucket_name}/{s3_path}')

# Ejemplo de uso
folder_path = 'Datos/Dog Emotion'
s3_folder_path = 'Datos/Dog Emotion'

upload_folder_to_s3(bucket, folder_path, s3_folder_path)

Subido Datos/Dog Emotion/labels.csv a s3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/labels.csv
Subido Datos/Dog Emotion/angry/0dWl0dRT9Pc98fjFSvGOXlGlE0B4E6267.jpg a s3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/angry/0dWl0dRT9Pc98fjFSvGOXlGlE0B4E6267.jpg
Subido Datos/Dog Emotion/angry/09dUVMcjCDfOtbeYDQg5Fvu3GPHWJg811.jpg a s3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/angry/09dUVMcjCDfOtbeYDQg5Fvu3GPHWJg811.jpg
Subido Datos/Dog Emotion/angry/0AvKtuzA7LfxnKaO0bey9mQMLnxXad73.jpg a s3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/angry/0AvKtuzA7LfxnKaO0bey9mQMLnxXad73.jpg
Subido Datos/Dog Emotion/angry/0C5yo7GxMy8lztxNZvSdfEx2gSPRTR701.jpg a s3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/angry/0C5yo7GxMy8lztxNZvSdfEx2gSPRTR701.jpg
Subido Datos/Dog Emotion/angry/0C9jEgFQHsh36W5U2u5CA98lB7C5eX806.jpg a s3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/angry/0C9jEgFQHsh36W5U2u5CA98lB7C5eX806.jpg
Subido Datos/Dog Emotion/angry/0RXraPIKC00

In [12]:

# Upload to S3 Bucket
from sagemaker import Session
prefix = 'Datos/Dog Emotion'

train_path = f"s3://{bucket}/{prefix}/train"
validation_path = f"s3://{bucket}/{prefix}/validation"
test_path = f"s3://{bucket}/{prefix}/test"
input_source = "s3://" + bucket+"/" + prefix



In [17]:
input_source

's3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion'

In [18]:
train_path

's3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/train'

In [19]:
test_path

's3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/test'

In [24]:
validation_path

's3://sagemaker-us-east-2-244680720635/Datos/Dog Emotion/validation'

In [43]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=get_execution_role(),
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='mlops-sklearnprocessing'
)


sklearn_processor.run(
    code='script.py',
    # arguments = ['arg1', 'arg2'],
    inputs=[
        ProcessingInput(
            source=input_source,
            destination="/opt/ml/processing/input",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train_data",
            source="/opt/ml/processing/output/train",
            destination=train_path,
        ),
        ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/output/validation", destination=validation_path),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test", destination=test_path),
    ]
)


.................Requirement already satisfied: pip in /miniconda3/lib/python3.7/site-packages (23.0)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0
    Uninstalling pip-23.0:
      Successfully uninstalled pip-23.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11 )                                                                                           │
│   12                                                                                             │
│   13                                                                                             │
│ ❱ 14 sklearn_processor.run(                                                                      │
│   15 │   code='script.py',                                                                       │
│   16 │   # arguments = ['arg1', 'arg2'],                                                         │
│   17 │   inputs=[                                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/processing.py:689 in run                       │
│                                                                                                  │
│    686 │   │   )                                                                                 │
│    687 │   │   self.jobs.append(self.latest_job)                                                 │
│    688 │   │   if wait:                                                                          │
│ ❱  689 │   │   │   self.latest_job.wait(logs=logs)                                               │
│    690 │                                                                                         │
│    691 │   def _include_code_in_inputs(self, inputs, code, kms_key=None):                        │
│    692 │   │   """Converts code to appropriate input and includes in input list.                 │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/processing.py:1114 in wait                     │
│                                                                                                  │
│   1111 │   │                                                                                     │
│   1112 │   │   """                                                                               │
│   1113 │   │   if logs:                                                                          │
│ ❱ 1114 │   │   │   self.sagemaker_session.logs_for_processing_job(self.job_name, wait=True)      │
│   1115 │   │   else:                                                                             │
│   1116 │   │   │   self.sagemaker_session.wait_for_processing_job(self.job_name)                 │
│   1117                                                     